In [ ]:
# This lecture is meant to show how great convolutions are and shows some examples of how they are derived from
# Applying some logic to linear regression.
# It will be followed up with a numpy session actually implementing these tools.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf

In [ ]:
from tensorflow.keras.datasets import cifar10

(trainX, trainy), (testX, testy) = cifar10.load_data()
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(trainX[i])

plt.show()

In [ ]:
n_data = 100
train_images = trainX[:n_data]
train_labels = pd.get_dummies(trainy[:n_data,0]).values
test_images = testX[:n_data]
test_labels = pd.get_dummies(testy[:n_data,0]).values
n_cls = train_labels.shape[1]

In [ ]:
tf.compat.v1.disable_eager_execution()

tfph = tf.compat.v1.placeholder
tfvar = tf.compat.v1.get_variable

In [ ]:
style = 'conv'

In [ ]:
tf.compat.v1.reset_default_graph()
if style == 'fc':
    img_ph = tfph(shape = (None, 32*32*3), dtype = tf.float32)
else:
    img_ph = tfph(shape = (None, 32, 32, 3), dtype = tf.float32)
features = img_ph
for i in range(3):
    if style == 'conv':
        features = tf.compat.v1.layers.conv2d(features, 64, (3, 3), padding = 'SAME')
    if style == 'lc':
        features = tf.keras.layers.LocallyConnected2D(4, (3, 3), padding = 'valid')(features)
    if style == 'fc':
        features = tf.compat.v1.layers.dense(features, 64 * 64 * 1)
    features = tf.nn.leaky_relu(features)
linear_features = tf.keras.layers.Flatten()(features)
yhat_raw = tf.compat.v1.layers.Dense(n_cls)(linear_features)
yhat = tf.nn.softmax(yhat_raw, axis=1)
y_true = tfph(shape = (None, n_cls), dtype = tf.float32)
ce_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_true, logits = yhat_raw, axis=1))
reg_loss = tf.reduce_sum([tf.reduce_sum(tf.square(v)) for v in tf.compat.v1.trainable_variables()])
loss = ce_loss + reg_loss * 0 # For now, disabling regularization.
opt = tf.compat.v1.train.AdamOptimizer().minimize(loss)
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [ ]:
batch_fd = {img_ph: train_images, y_true: train_labels}
if style == 'fc':
    batch_fd[img_ph] = train_images.reshape(-1, 32*32*3)

In [ ]:
for i in range(1000):
  _, cur_loss = sess.run(
      [opt, loss], batch_fd)
  if i % 10 == 0:
    print(cur_loss)